In [45]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict

In [46]:
class PrivateState(TypedDict):
    a: int
    b: int


class InputState(TypedDict):
    hello: str


class OutputState(TypedDict):
    bye: str


class MegaPrivate(TypedDict):
    secret: bool


graph_builder = StateGraph(
    PrivateState,
    input_schema=InputState,
    output_schema=OutputState,
)

In [ ]:
def node_one(state: InputState) -> InputState:
    print("node_one ->", state)
    return {"hello": "world"}

# node_one에서 출력된 InputState는 무시
def node_two(state: PrivateState) -> PrivateState:
    print("node_two ->", state)
    return {
        "a": 1,
    }


def node_three(state: PrivateState) -> PrivateState:
    print("node_three ->", state)
    return {"b": 1}


def node_four(state: PrivateState) -> OutputState:
    print("node_four ->", state)
    return {
        "bye": "world",
    }


def node_five(state: OutputState):
    return {"secret": True}


def node_six(state: MegaPrivate):
    print(state)

In [48]:
graph_builder.add_node("node_one", node_one)
graph_builder.add_node("node_two", node_two)
graph_builder.add_node("node_three", node_three)
graph_builder.add_node("node_four", node_four)
graph_builder.add_node("node_five", node_five)
graph_builder.add_node("node_six", node_six)

graph_builder.add_edge(START, "node_one")
graph_builder.add_edge("node_one", "node_two")
graph_builder.add_edge("node_two", "node_three")
graph_builder.add_edge("node_three", "node_four")
graph_builder.add_edge("node_four", "node_five")
graph_builder.add_edge("node_five", "node_six")
graph_builder.add_edge("node_six", END)

In [59]:
graph = graph_builder.compile()


result = graph.invoke(
    {"hello": "world"},
)

print(result)

node_one -> {'hello': 'world'}
node_two -> {}
node_three -> {'a': 1}
node_four -> {'a': 1, 'b': 1}
{'secret': True}
{'bye': 'world'}


In [ ]:
# print(graph.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	node_one(node_one)
	node_two(node_two)
	node_three(node_three)
	node_four(node_four)
	node_five(node_five)
	node_six(node_six)
	__end__([<p>__end__</p>]):::last
	__start__ --> node_one;
	node_five --> node_six;
	node_four --> node_five;
	node_one --> node_two;
	node_three --> node_four;
	node_two --> node_three;
	node_six --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

